# Observation Model Test

This notebook downloads an image and processes it with an observation model from the `transformers` library.

In [ ]:
import requests
from PIL import Image
from transformers import AutoImageProcessor, AutoModel

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained(
    "apple/aimv2-large-patch14-224-distilled",
)
model = AutoModel.from_pretrained(
    "apple/aimv2-large-patch14-224-distilled",
    trust_remote_code=True,
)
inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)
